<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [2]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.8
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2
#количество входов/выходов соответственно
countX = 2
countY = 2

networkCharact = [layer, neurons, learningRate]
inpOutCounts = [countX, countY]

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [3]:
#charactNet = [кол-во слоёв, кол-во нейронов, коэф. обучения]
#sizeSystem = [кол-во входов, кол-во выходов]


#random.sample(data, len(data) - 1) --- случайный выбор индекса

def deltaRule(function, data, charactNet, sizeSystem):

  si = [0] * sizeSystem[1]
  yi = [0] * sizeSystem[1]
  epsi = [0] * sizeSystem[1]
  level_i = 0
  k = 0

  while(level_i < charactNet[0]): #цикл по слоям
    wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(charactNet[1])] #синаптические веса

    print("\tЭпоха №" + str(level_i + 1))

    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = ( " + str(data[data.columns[0]][k]) + ", " + str(data[data.columns[1]][k]) + ' )')

      for i in range(charactNet[1]): #по нейронам
        for j in range(sizeSystem[0]): #по входам
          si[i] += data[data.columns[j]][k] * wi[j][i]

        #сделать проверку на сравнение количества нейронов и выходов для последнего слоя!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        yi[i] = function(si[i])
        epsi[i] = data[data.columns[sizeSystem[1] + i]][k] - yi[i]
        eps += epsi[i]**2

      rez_i = pd.DataFrame([si, yi, epsi], index=['S', 'Y расч.', 'EPS'], columns=['neuron ' + str(i+1) for i in range(sizeSystem[1])])
      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(charactNet[1]):
        for j in range(sizeSystem[0]):
          wi[j][i] = wi[j][i] - charactNet[2] * epsi[i] * data[data.columns[j]][k]

      k += 1

    level_i += 1
    k = 0

  #

In [ ]:
table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(hyperbolicTangentFunction, table, networkCharact, inpOutCounts)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

	Эпоха №1
Матрица весов для 1-й итерации:
          0         1
0  0.797455  0.042986
1  0.887484  0.548104

Для X = ( 0, 0 )
         neuron 1  neuron 2
S             0.0       0.0
Y расч.       0.0       0.0
EPS           1.0       0.0

СКО: 1.0


Матрица весов для 2-й итерации:
          0         1
0  0.797455  0.042986
1  0.887484  0.548104

Для X = ( 0, 1 )
         neuron 1  neuron 2
S        0.887484  0.548104
Y расч.  0.416741  0.267391
EPS     -0.416741 -0.267391

СКО: 0.24517137002763426


Матрица весов для 3-й итерации:
          0         1
0  0.797455  0.042986
1  1.220877  0.762017

Для X = ( 1, 0 )
         neuron 1  neuron 2
S        1.684939  0.591091
Y расч.  0.687115  0.287231
EPS     -0.687115 -0.287231

СКО: 0.5546282899490129


Матрица весов для 4-й итерации:
          0         1
0  1.347147  0.272771
1  1.220877  0.762017

Для X = ( 1, 1 )
         neuron 1  neuron 2
S        4.252963  1.625879
Y расч.  0.971955  0.671209
EPS      0.028045  0.328791

СКО: 0.108

In [ ]:
#1-Количество входов и выходов могут быть различны +
#2-Реализовать расчёт промежуточных слоёв

In [ ]:
#Характеристики многослойной сети:
#1- вектор из кол-ва нейронов в каждом слое (в каждом слое м/б разное кол-во нейронов)
#2- кол-во выходов = кол-ву нейронов в крайнем слое
#3- входов может быть сколько угодно +
#4- в цикле: wij: i-номер нейрона, j-номер входа +
#5- размер s, eps равен количеству выходов +

In [ ]:
#1-Добавить условие для последнего слоя, где сравнивается кол-во нейронов и кол-во выходов
#2-

In [16]:
#для случайного отбора строк данных в модель
tt = [[1,2], [3,4], [5,6], [5,6], [5,6], [5,6], [5,6], [5,6], [5,6], [5,6], [5,6], [5,6]]
rr = [i for i in range(0, len(tt))]
random.shuffle(rr)
rr

[0, 9, 3, 1, 11, 2, 10, 5, 6, 4, 7, 8]